# Deep Learning
---
Это нужно знать:
 1. Аугментация данных
 2. Предобработка данных
 3. Инициализация
 4. Процесс обучения
 5. Выбор функции активации
 6. Визуализация



## 1. Аугментация данных
**Аугментация данных** - это "раздутие" датасета для тренировки нейроной сети

### Создание обучающих  примеров из искусственных изображений
Можно взять несколько шаблонов / «идеальных» примеров (например, наборов шрифтов) и с помощью различных искажений создать необходимое число примеров для обучения. Можно использовать следующие искажения:
1. Геометрические (афинные, проективные, ...).
2. Яркостные/цветовые.
3. Замена фона.
4. Искажения, характерные для решаемой задачи: блики, шумы, размытие и т. д.

**Сдвиги:**

![](img/sdvigi.png)

**Повороты:**

![](img/povoroty.png)

**Дополнительные линии на изображениях:**

![](img/doplin.png)

**Блики:**

![](img/bliki.png)

**Дефокус:**

![](img/defocus.png)

**Сжатия и растяжения вдоль осей:** 

![](img/szhatie.png)

## 2. Предобработка данных
Итак, мы получили большое количество обучающих изображений. Теперь необходимо выполнить их предобработку. Далее мы рассмотрим несколько вариантов этой процедуры.

Одним из наиболее распространенных методов предобработки является центрирование данных к нулю (zero-centering) и последующая нормализация (normalization). Для этого понадобится две строки кода на Python:
```python
X -= np.mean(X, axis = 0) # zero-center
X /= np.std(X, axis = 0) # normalize
```
где X представляет собой массив входных данных (количество элементов × количество измерений). Другая форма этой предобработки нормализует каждое измерение таким образом, чтобы минимальное и максимальное значение по данному измерению были равны –1 и 1 соответственно. Данную предобработку имеет смысл применять только в том случае, когда есть основания полагать, что различные входные признаки имеют различный масштаб (или единицы измерения), но при этом обладают примерно одинаковой ценностью для модели. В случае изображений, относительные масштабы пикселей исходно примерно одинаковы (и находятся в интервале от 0 до 255), поэтому нет строгой необходимости выполнять данную предобработку.

Другой метод предобработки, аналогичный первому, называется PCA отбеливание (PCA whitening). Данные сначала центрируются, как описано выше, а затем вычисляется ковариационная матрица, описывающая структуру корреляций в данных:
```python
X -= np.mean(X, axis = 0) # zero-center
X /= cov = np.dot(X.T, X) / X.shape[0] # compute the covariance matrix
```
Далее мы выполняем декорреляцию, проецируя исходные данные (центрированные) на собственный базис (eigenbasis):
```python
U,S,V = np.linalg.svd(cov) # compute the SVD factorization of the data covariance matrix
Xrot = np.dot(X, U) # decorrelate the data
```
Последним этапом является отбеливание. Мы берем данные в собственном базисе и делим каждое измерение на собственное значение, чтобы нормализовать масштаб:
```python
Xwhite = Xrot / np.sqrt(S + 1e-5) # divide by the eigenvalues (which are square roots of the singular values)
```
Обратите внимание, мы прибавляем малую константу 1e–5, чтобы предотвратить деление на ноль. Недостатком этого преобразования является то, что в его результате может быть значительно усилен шум в данных. Это связано с тем, что данный метод преобразует все измерения к одинаковому размеру, в том числе, измерения, обладающие малой дисперсией и являющиеся в своем большинстве шумом. На практике этот недостаток можно нивелировать с помощью более сильного сглаживания (т.е. увеличив значение малой константы).

Для полноты мы рассмотрели несколько методов предобработки. Следует отметить, что в случае сверточных сетей последний метод не применяется, в то время как центрирование данных является необходимым. Кроме того, часто выполняется нормализация каждого пикселя.

![](img/ex.png)

## 3. Инициализация
Данные готовы, но прежде чем начать обучение сети, необходимо инициализировать ее параметры.

### Инициализация малыми случайными значениями
Таким образом, нам нужно, чтобы веса были близки к нулю, но не равны ему. Для этого мы можем инициализировать их малыми случайными значениями очень близкими к нулю, что позволит нарушить симметрию. В результате, все исходные веса будут случайными и уникальными, следовательно, обновляться они будут по-разному, что нам и нужно. Вычислить веса можно следующим образом:

*weights ~ 0,001 × N(0, 1)*

где *N(0, 1)* – нормальное распределение с математическим ожиданием, равным 0, и среднеквадратическим отклонением, равным 1. Кроме того, можно использовать малые случайные значения из равномерного распределения, но на практике этот подход не оказывает существенного влияния на результат.

## 4. Процесс обучения

Итак, все готово. Приступим к обучению глубоких сетей!

### Фильтры и размер объединения (pooling size)

Желательно, чтобы размер входных изображений представлял собой степень двойки, например, 32 (CIFAR-10), 64, 224 (ImageNet), 384, 512 и т.д. Кроме того, важно применять малый фильтр (например, 3 × 3) и малый шаг (stride) (например, 1) с заполнением нулями (zero-padding),

![](img/ex1.png)

что позволяет не только уменьшить количество параметров, но и повысить точность сети в целом. Частный случай, упомянутый выше (фильтр 3 × 3 с шагом 1), позволяет сохранить пространственный размер карт признаков и изображений. Рекомендуемый размер объединения составляет 2 × 2.

## 5. Выбор функции активации
Одним из важнейших аспектов глубокой нейронной сети является функция активации (activation function), которая привносит в сеть нелинейность. Далее мы рассмотрим распространенные функции активации и дадим рекомендации по их выбору.

![](img/neuron.png)

### Гиперболический тангенс

![](img/thx.png)

Гиперболический тангенс (hyperbolic tangent, tanh) принимает на входе произвольное вещественное число, а на выходе дает вещественное число в интервале от –1 до 1.

Формула гиперболического тангенса:
             <p><h1>$th(x)=\frac{e^x+e^{-x}}{e^x-e^{-x}}$</h1></p>

## ReLU
![](img/relu.png)

В последние годы большую популярность приобрела функция активации под названием «выпрямитель» (rectifier, по аналогии с однополупериодным выпрямителем в электротехнике). Нейроны с данной функцией активации называются ReLU (rectified linear unit). ReLU имеет следующую формулу f(x) = max(0, x) и реализует простой пороговый переход в нуле.

В настоящее время существует целое семейство различных модификаций ReLU:

![](img/relus.png)